# Explaination about this code
In this code I translated all the text from the news websites to english. This was nececary because the sentiment analysis models that I used did not support the german language.

**Caution!** This code will take about 4:40h. 

For further reading please look at: `capstone.ipynb`!

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from deep_translator import GoogleTranslator, LibreTranslator, DeeplTranslator, MyMemoryTranslator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from pathlib import Path
import os
import time
from tqdm import tqdm

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/generalpegasus/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [25]:
# ..\\data\\DAX-News_n=455.csv'
file = Path(os.path.join('..', 'data', 'news', 'all_news_text.csv'))
df = pd.read_csv(file, index_col=0)
display(df.head(3))
df.shape

,date,news_text,headline,article_url
0,17.10.24 09:16 Uhr,FRANKFURT (dpa-AFX) - DerDAXist am Tag der EZB...,Aktien Frankfurt Eröffnung: Dax steigt vor EZB...,https://www.finanzen.net/nachricht/aktien/akti...
1,17.10.24 08:26 Uhr,FRANKFURT (dpa-AFX) - Am Tag der Zinsentscheid...,Aktien Frankfurt Ausblick: Dax vor Zinsentsche...,https://www.finanzen.net/nachricht/aktien/akti...
2,17.10.24 08:00 Uhr,Behalten Sie hier den kompletten aktuellen Han...,Was heute an der Börse wichtig ist: EZB entsch...,https://www.finanzen.net/nachricht/aktien/10-v...


(16906, 4)

In [26]:

# drop rows which contain 'None'
df = df.dropna(subset=['date', 'news_text'])

# tranform the date to datetime
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%y %H:%M Uhr')

# Get just the date without 'Uhr'
df['date'] = df['date'].dt.date

# sort the values by date ascending
df.sort_values(by='date', ascending=True, inplace=True)

display(df.head(3))
print(df.shape)

,date,news_text,headline,article_url
16905,2017-10-18,FRANKFURT (Dow Jones)--Die Deutsche Börse zieh...,Deutsche Börse will Anfang 2018 Scale-Auswahli...,https://www.finanzen.net/nachricht/aktien/deut...
16899,2017-10-18,WIEN (dpa-AFX) - Die Wiener Börse hat am Mittw...,Aktien Wien Schluss: ATX erstmals seit 2008 üb...,https://www.finanzen.net/nachricht/aktien/akti...
16900,2017-10-18,FRANKFURT (dpa-AFX) - Die Anleger am deutschen...,ROUNDUP/Aktien Frankfurt Schluss: Wall Street ...,https://www.finanzen.net/nachricht/aktien/roun...


(16905, 4)


In [27]:
print(df.news_text.isnull().sum())
print(df.news_text.isna().sum())
df['news_text'] = df['news_text'].astype(str)

0
0


In [28]:
import time
import pandas as pd
from deep_translator import GoogleTranslator

# Funktion, um den Text in kleinere Abschnitte aufzuteilen
def split_text(text, max_length=4999):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# Übersetzungsfunktion
def translate_text(text, max_retries=5, wait_time=120):
    parts = split_text(text)

    # List of translators to try in order
    translators = [
        GoogleTranslator(source='de', target='en'),
        # LibreTranslator(source='de', target='en', api_key=None, use_free_api=True),
        # DeeplTranslator(source='de', target='en', api_key='your_deepl_api_key'),
        # MyMemoryTranslator(source='de-DE', target='en-US')
    ]
    
    for trial in range(max_retries):
        for translator in translators:
            try:
                translated_parts = [translator.translate(part) for part in parts]
                # Ensure all parts are valid strings
                if all(isinstance(part, str) for part in translated_parts):
                    return ' '.join(translated_parts)
            except Exception as e:
                print(f"Error with {translator.__class__.__name__}: {e}")
        
        print(f"Trial {trial + 1} failed. Waiting {wait_time} seconds before trying again.")
        time.sleep(wait_time)
    
    return 'Translation failed'

for i, idx in tqdm(enumerate(df['news_text']), desc='Translating', total=df.shape[0]):
    df.loc[i, 'en_news_text'] = translate_text(idx)


# Zeige die ersten 3 Zeilen an
display(df.head(3))
# This code was running ofr 4h 14min

Translating:   0%|          | 2/16905 [00:00<1:06:54,  4.21it/s]

Translating:  10%|▉         | 1660/16905 [17:11<4:40:20,  1.10s/it]

Error with GoogleTranslator: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Trial 1 failed. Waiting 120 seconds before trying again.


Translating:  53%|█████▎    | 9015/16905 [1:39:43<1:11:06,  1.85it/s]

Trial 1 failed. Waiting 120 seconds before trying again.
Trial 2 failed. Waiting 120 seconds before trying again.
Trial 3 failed. Waiting 120 seconds before trying again.
Trial 4 failed. Waiting 120 seconds before trying again.
Trial 5 failed. Waiting 120 seconds before trying again.


Translating: 100%|██████████| 16905/16905 [4:14:10<00:00,  1.11it/s]    


,date,news_text,headline,article_url,en_news_text
16905,2017-10-18,FRANKFURT (Dow Jones)--Die Deutsche Börse zieh...,Deutsche Börse will Anfang 2018 Scale-Auswahli...,https://www.finanzen.net/nachricht/aktien/deut...,NaN
16899,2017-10-18,WIEN (dpa-AFX) - Die Wiener Börse hat am Mittw...,Aktien Wien Schluss: ATX erstmals seit 2008 üb...,https://www.finanzen.net/nachricht/aktien/akti...,FRANKFURT (dpa-AFX) - After a continued record...
16900,2017-10-18,FRANKFURT (dpa-AFX) - Die Anleger am deutschen...,ROUNDUP/Aktien Frankfurt Schluss: Wall Street ...,https://www.finanzen.net/nachricht/aktien/roun...,Keep an eye on the entire current trading day ...


In [29]:
df.to_csv(Path(os.path.join('..', 'data', 'news', 'all_news_text_en.csv')))

For further reading please look at: `capstone.ipynb`!